# Analiza danych przestrzennych - ćwiczenia laboratoryjne 2022/2023

---

## Zestaw zadań 3: Badanie intensywności procesów punktowych (część 2)

In [1]:
import numpy as np
import pandas as pd
import scipy as sp
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Miejsce do importu pakietów wchodzących w skład standardowej biblioteki Pythona oraz ustawienie opcji wykorzystywanych pakietów
sns.set() 
sns.set_theme(style="whitegrid")

In [3]:
# Miejsce do wklejenie funkcji ze wcześniejszych zestawów zadań
def homogeneous_poisson_on_rectangle(intensity, x_lim, y_lim):
    """
    Parameters
    -------
    intensity: float
        Liczba dodatnia określająca intensywność procesu punktowego.
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   
    
    Returns
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    """
    n = np.random.poisson(intensity*(x_lim[1]-x_lim[0])*(y_lim[1]-y_lim[0]))
    x = np.random.uniform(x_lim[0], x_lim[1], n)
    y = np.random.uniform(y_lim[0], y_lim[1], n)
    return pd.DataFrame({"X" : x, "Y" : y})

def point_count_on_subregions(points, bins, x_lim, y_lim):
    """
    Parameters
    -------
    points: DataFrame
        Tablica zawierająca dwie kolumny ze współrzędnymi punktów opisane jako "X" i "Y".
    bins: list
        Lista określająca liczbę podobszarów w poziomie i pionie.
        Przykład: [10, 10]
    x_lim: list
        Lista określająca zakres wartości współrzędnej X.
        Przykład: [0, 10]
    y_lim: list
        Lista określająca zakres wartości współrzędnej Y.
        Przykład: [0, 10]   

    Returns
    -------
    bin_data: list
        Lista zawierająca trzy macierze:
        - 1D ze współrzędnymi krawędzi podobszarów na osi X,
        - 1D ze współrzędnymi krawędzi podobszarów na osi Y,
        - 2D z liczbą punków przypisanych do każdego z podobszarów.
        Na przykład: [array([0, 1, 2]), array([0, 1, 2]), array([[7, 2], [4, 5]])]
    """
    hist, x, y = np.histogram2d(points["X"], points["Y"], bins = bins, range = [x_lim, y_lim])
    return [x, y, np.transpose(hist)]

### Przygotowanie danych
Wczytaj dane zawarte w plikach CSV załączonych do zestawu zadań.

In [4]:
df1 = pd.read_csv("points_1.csv")
df2 = pd.read_csv("points_2.csv")
df3 = pd.read_csv("points_3.csv")

### Zadanie 1: Test chi-kwadrat Pearsona (30 pkt)

Przygotuj funkcję `pearsons_chi2_test()`, która będzie przeprowadzać test istotności chi-kwadrat Pearsona i wyświetlać jego wynik zgodnie z pokazanym poniżej schematem. Następnie wykorzystaj przygotowanią funkcję do sprawdzenia, czy rozkłady punktowe zaimportowane z plików points_1.csv i points_2.csv są jednorodnymi rozkładami Poissona o intensywności równej 20. W obliczeniach przyjmij $\alpha=0.05$.

Rozwiązanie zadania wymaga dodatkowo przygotowania funkcji pomocniczych `distribution_table()` i `poisson_distribution_table()`, które będą przygotowywać szeregi rodzielcze testowanego rozkładu oraz teoretycznego rozkładu Poissona.

Algorytm postępowania:
- Formułujemy hipotezę zerową i hipotezę alternatywną H1: <br/>
H0: Testowana zmienna ma przyjęty rozkład teoretyczny <br/>
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
- Obliczamy wartość statystyki testowej $\chi^2$: <br/>
$\chi^2 = \sum_{i=1}^{k} \frac{(n_i-n p_i)^2}{np_i}$ <br/>
gdzie: $k$ - liczba wariantów badanej cechy, $n_i$ - liczebność i-tego wariantu testowanego rozkładu, $n$ - liczba punktów testowanego rozkładu, $p_i$ - prawdopodobieństwo  i-tego wariantu rozkładu teoretycznego.
- Z rozkładu chi-kwadrat wyznaczamy obszar krytyczny testu istotności $\chi^2_{\alpha}$: <br/>
$\chi^2_{\alpha} = \chi^2_{1-\alpha, k-s-1}$ <br/>
gdzie:  $\alpha$ - poziom istotności, $k$ - liczba wariantów rozkładu, $s$ - liczba nieznanych parametrów rozkładu.
- Podejmujemy decyzję weryfikującą: <br/>
$\chi^2 >= \chi^2_{\alpha}$ - Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = X <br/>
$\chi^2 < \chi^2_{\alpha}$ - Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = X

Przykładowe wyniki pracy funkcji `pearsons_chi2_test()`: <br/>
<br/>
`Test chi-kwadrat Pearsona` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`chi2 = 23.307 chi2_alpha = 18.078`<br/>
`chi2 >= chi2_alpha` <br/>
`Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>
<br/>
`Test chi-kwadrat Pearsona` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`chi2 = 19.521 chi2_alpha = 21.129`<br/>
`chi2 < chi2_alpha` <br/>
`Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>

#### a) Przygotowanie funkcji

In [5]:
def distribution_table(bin_counts):
    """
    Parameters
    -------
    bin_counts: array
        Macierz 2D z liczbą punków przypisanych do każdego z podobszarów.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera wszystkie wartości całkowite z zakresu od minimalnej do maksymalnej liczby zliczeń w obrębie podobszarów,
        - "N(K)", która zawiera liczby podobszarów, którym zostały przypisane poszczególne liczby punktów.
    """
    binFlat = bin_counts.flatten()
    K = np.arange(binFlat.min(), binFlat.max()+1, dtype=int)
    NK = np.array([np.sum(binFlat == x) for x in K])
    return pd.DataFrame({"K" : K, "N(K)" : NK})

def poisson_distribution_table(k, mu):
    """
    Parameters
    -------
    k: array
        Macierz 1D z wariantami badanej cechy, dla którym ma zostać wyliczone prawdopodobieństwo.
    mu: int
        Wartość oczekiwana rozkładu Poissona.

    Returns
    -------
    table: DataFrame
        Tablica zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy,
        - "P(K)", która zawiera wartości prawdopodobieństw rozkładu Poissona wyliczone dla wartości oczekiwanej mu
        oraz poszczególnych wariantów badanej cechy znormalizowane do sumy wartości równej 1.
    """  
    return pd.DataFrame({"K" : k, "P(K)" : sp.stats.poisson.pmf(k, mu)})

def pearsons_chi2_test(tested_distribution, theoretical_distribution, alpha, ddof):
    """
    Parameters
    -------
    tested_distribution: DataFrame
        Tablica opisująca testowany rozkład i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej theoretical_distribution,
        - "N(K)", która zawiera liczebności poszczególnych wariantów badanej cechy.

    theoretical_distribution: DataFrame
        Tablica opisująca rozkład teoretyczny i zawierająca 2 kolumny:
        - "K", która zawiera warianty badanej cechy, wartości muszą być identycznej jak kolumna "K" zmiennej lokalnej tested_distribution,
        - "P(K)", która zawiera prawdopodobieństwa poszczególnych wariantów badanej cechy. Wartości z tej kolumny muszą sumować się do 1.
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """
    N = tested_distribution["N(K)"].to_numpy()
    n = np.sum(N)
    P = theoretical_distribution["P(K)"].to_numpy()
    chi2 = np.sum((N-n*P)**2/(n*P))
    chi2a = sp.stats.chi2.ppf(1-alpha, len(tested_distribution)-ddof-1)
    
    if chi2 >= chi2a:
        return """Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = {:.3f} chi2_alpha = {:.3f}
chi2 >= chi2_alpha
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = {}""".format(chi2, chi2a, alpha)
    return """Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = {:.3f} chi2_alpha = {:.3f}
chi2 < chi2_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {}""".format(chi2, chi2a, alpha)

#### b) Weryfikacja hipotezy o rozkładzie 1

In [6]:
x, y, hist = point_count_on_subregions(df1, [40, 20], [0, 20], [0, 10])
tested_distribution = distribution_table(hist)
theoretical_distribution = poisson_distribution_table(tested_distribution["K"], 20*(x[1]-x[0])*(y[1]-y[0]))
res = pearsons_chi2_test(tested_distribution, theoretical_distribution, 0.05, 0)
print(res)

Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 17.788 chi2_alpha = 22.362
chi2 < chi2_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05


#### c) Weryfikacja hipotezy o rozkładzie 2

In [7]:
x, y, hist = point_count_on_subregions(df2, [40, 20], [0, 20], [0, 10])
tested_distribution = distribution_table(hist)
theoretical_distribution = poisson_distribution_table(tested_distribution["K"], 20*(x[1]-x[0])*(y[1]-y[0]))
res = pearsons_chi2_test(tested_distribution, theoretical_distribution, 0.05, 0)
print(res)

Test chi-kwadrat Pearsona
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
chi2 = 86.340 chi2_alpha = 22.362
chi2 >= chi2_alpha
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05


### Zadanie 2: Test Kołmogorowa - Smirnowa (20 pkt)

Przygotuj funkcję `kolmogorow_smirnow_test()`, która będzie przeprowadzać test istotności Kołmogorowa-Smirnowa i wyświetlać jego wynik zgodnie z pokazanym poniżej schematem. Następnie wykorzystaj przygotowanią funkcję do sprawdzenia, czy rozkład punktowy zaimportowany z pliku points_3.csv jest jednorodnym rozkładem Poissona. W obliczeniach przyjmij poziom istotności $\alpha=0.05$.

Algorytm postępowania:
- Formułujemy hipotezę zerową i hipotezę alternatywną H1: <br/>
H0: Testowana zmienna ma przyjęty rozkład teoretyczny <br/>
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
- Obliczamy wartość statystyki testowej $\lambda$: <br/>
$D = \sup_{x}(|F_t - F_0|)$ <br/>
$\lambda = D\sqrt{n}$ <br/>
gdzie: $F_t$ - dystrybuanta testowanego rozkładu,  $F_0$ - dystrybuanta rozkładu teoretycznego, $n$ - liczba punktów.
- Z rozkładu Kołomogorowa wyznaczamy obszar krytyczny testu istotności $\lambda_{\alpha}$: <br/>
$\lambda_{\alpha} = \lambda_{1-\alpha}$ <br/>
gdzie:  $\alpha$ - poziom istotności.
- Podejmujemy decyzję weryfikującą: <br/>
$\lambda >= \lambda_{\alpha}$ - Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = X <br/>
$\lambda < \lambda_{\alpha}$ - Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = X

Uwaga! Test należy przeprowadzić niezależnie dla współrzędnej X i Y. Decyzja jest podejmowana na podstawie wyników obu testów.


Przykładowe wyniki pracy funkcji `kolmogorow_smirnow_test()`: <br/>
<br/>
`Test Kołmogorowa-Smirnowa dla współrzędnej X` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`lambda = 2.036  lambda_alpha = 1.255`<br/>
`lambda >= lambda_alpha` <br/>
`Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05` <br/>
<br/>
`Test Kołmogorowa-Smirnowa dla współrzędnej Y` <br/>
`H0: Testowana zmienna ma przyjęty rozkład teoretyczny` <br/>
`H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego` <br/>
`lambda = 1.136  lambda_alpha = 1.748` <br/>
`lambda < D_alpha` <br/>
`Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05`

#### a) Przygotowanie funkcji

In [8]:
def kolmogorow_smirnow_test(tested_points, theoretical_points, alpha, ddof):
    """
    Parameters
    -------
    tested_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów testowanego rozkładu opisaną jako "X" lub "Y".

    theoretical_points: DataFrame
        Tablica zawierająca kolumnę ze współrzędnymi punktów toeretycznego rozkładu opisaną jako "X" lub "Y".
    
    alpha: float
        Wartość z zakresu [0,1] określająca poziom istotności.
    
    ddof: int
        Liczba nieujemna określająca liczbę nieznanych parametrów rozkładu.
    """   
    Dn, p_val = sp.stats.kstest(tested_points, theoretical_points, alternative="two_sided")
    n = len(tested_points)
    lambd = Dn*(n**0.5)
    lambd_alpha = sp.stats.kstwobign.ppf(1-alpha)
    
    if lambd >= lambd_alpha:
        return """Test Kołmogorowa-Smirnowa dla współrzędnej {}
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = {:.3f} lambda_alpha = {:.3f}
lambda >= lambda_alpha
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = {}""".format(tested_points.name, lambd, lambd_alpha, alpha) 
    return """Test Kołmogorowa-Smirnowa dla współrzędnej {}
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = {:.3f} lambda_alpha = {:.3f}
lambda < lambda_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = {}""".format(tested_points.name, lambd, lambd_alpha, alpha) 

#### b) Weryfikacja hipotezy o rozkładzie 3

In [9]:
df_theor = pd.DataFrame({"X" : np.linspace(df3["X"].min(), df3["X"].max(), len(df3["X"])), 
                         "Y" : np.linspace(df3["Y"].min(), df3["Y"].max(), len(df3["Y"]))})
res1 = kolmogorow_smirnow_test(df3['X'], df_theor['X'], 0.05, 0)
res2 = kolmogorow_smirnow_test(df3['Y'], df_theor['Y'], 0.05, 0)
print(res1)
print(res2)

Test Kołmogorowa-Smirnowa dla współrzędnej X
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 2.156 lambda_alpha = 1.358
lambda >= lambda_alpha
Odrzucenie H0 na rzecz H1 na poziomie istotności alpha = 0.05
Test Kołmogorowa-Smirnowa dla współrzędnej Y
H0: Testowana zmienna ma przyjęty rozkład teoretyczny
H1: Testowana zmienna nie ma przyjętego rozkładu teoretycznego
lambda = 0.571 lambda_alpha = 1.358
lambda < lambda_alpha
Wynik testu istotności nie daje podstaw do odrzucenia H0 na rzecz H1 na poziomie istotności alpha = 0.05
